<a href="https://colab.research.google.com/github/Ruchira1214/CodTech_ML_tasks/blob/main/task3_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1 — Check GPU
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

PyTorch version: 2.8.0+cu126
CUDA available: False
Device: cpu


In [2]:
# Cell 2 — Install sklearn (runs fast)
!pip install -q scikit-learn

In [3]:
# Cell 3 — Fast PyTorch CNN training on CIFAR-10 (paste whole cell)
import os, time
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 128
NUM_EPOCHS = 5       # small for quick run
LR = 1e-3
SAVE_PATH = "/content/cnn_model.pth"
NUM_WORKERS = 2

# --- Simple (fast) CNN ---
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),  # 32x16x16
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2), # 64x8x8
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2) # 128x4x4
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*4*4, 256),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        x = self.conv(x)
        x = self.classifier(x)
        return x

# --- Data ---
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.247,0.243,0.261))
])
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.247,0.243,0.261))
])

train_set = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform_train)
test_set  = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform_test)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_loader  = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)
classes = train_set.classes

# --- Model / Loss / Opt ---
model = SimpleCNN(num_classes=len(classes)).to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

# --- Training loop ---
def train_one_epoch(model, loader, opt, criterion):
    model.train()
    running_loss = 0.0; correct = 0; total = 0
    for images, labels in loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        opt.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        opt.step()
        running_loss += loss.item()*images.size(0)
        preds = outputs.argmax(dim=1)
        correct += (preds==labels).sum().item()
        total += labels.size(0)
    return running_loss/total, correct/total

def eval_model(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    preds_all, labels_all = [], []
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()*images.size(0)
            preds_all.append(outputs.argmax(dim=1).cpu().numpy())
            labels_all.append(labels.cpu().numpy())
    preds_all = np.concatenate(preds_all)
    labels_all = np.concatenate(labels_all)
    return running_loss/len(labels_all), preds_all, labels_all

start_time = time.time()
best_acc = 0.0
for epoch in range(1, NUM_EPOCHS+1):
    t0 = time.time()
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, preds, labels = eval_model(model, test_loader, criterion)
    val_acc = (preds==labels).mean()
    epoch_time = time.time()-t0
    print(f"Epoch {epoch}/{NUM_EPOCHS} — {epoch_time:.1f}s  Train loss: {train_loss:.4f} Acc: {train_acc:.4f}  |  Test loss: {val_loss:.4f} Acc: {val_acc:.4f}")
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), SAVE_PATH)
print(f"\nDone in {(time.time()-start_time)/60:.2f} minutes. Best test acc: {best_acc:.4f}")
# --- Final metrics ---
from sklearn.metrics import classification_report, confusion_matrix
_, preds, labels = eval_model(model, test_loader, criterion)
print("\nClassification report:\n", classification_report(labels, preds, target_names=classes, zero_division=0, digits=4))
print("Confusion matrix shape:", confusion_matrix(labels, preds).shape)
print(f"\nSaved best model to: {SAVE_PATH}")


100%|██████████| 170M/170M [00:03<00:00, 42.7MB/s]


Epoch 1/5 — 122.7s  Train loss: 1.6357 Acc: 0.3958  |  Test loss: 1.2450 Acc: 0.5533
Epoch 2/5 — 118.9s  Train loss: 1.2468 Acc: 0.5503  |  Test loss: 0.9849 Acc: 0.6510
Epoch 3/5 — 118.1s  Train loss: 1.0746 Acc: 0.6191  |  Test loss: 0.8793 Acc: 0.6915
Epoch 4/5 — 118.0s  Train loss: 0.9578 Acc: 0.6639  |  Test loss: 0.8214 Acc: 0.7159
Epoch 5/5 — 117.5s  Train loss: 0.8914 Acc: 0.6892  |  Test loss: 0.7621 Acc: 0.7318

Done in 9.92 minutes. Best test acc: 0.7318

Classification report:
               precision    recall  f1-score   support

    airplane     0.7459    0.7750    0.7602      1000
  automobile     0.8801    0.8440    0.8617      1000
        bird     0.5866    0.6500    0.6167      1000
         cat     0.5979    0.4490    0.5128      1000
        deer     0.7367    0.6490    0.6901      1000
         dog     0.6084    0.7100    0.6553      1000
        frog     0.8643    0.7260    0.7891      1000
       horse     0.6982    0.8260    0.7568      1000
        ship     0

In [4]:
# Cell 4 — Download the saved model to your computer (runs in Colab)
from google.colab import files
files.download("/content/cnn_model.pth")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>